## Analyse Appliences on SmartMeters in Buildings 
### UKDALE2 Dataset

In [4]:
import nilmtk

start_time = '2015-01-01'
end_time = '2015-04-01'
sample_period = 10


ukdale = nilmtk.DataSet("ukdale2.h5")
ukdale.set_window(start=start_time, end=end_time)


arr = []
for j in range(len(ukdale.buildings)):
    arr2 = []
    for i in range(len(ukdale.buildings[j+1].elec.appliances)):
        arr2.append(ukdale.buildings[1].elec.appliances[i].type["type"])
    arr.append(arr2)

common_apps = set(arr[0]).intersection(arr[1], arr[4]) # Übereinstimmungen aus Building 1, 2 und 5
print(common_apps)

{'hair straighteners', 'toaster', 'light', 'wireless phone charger', 'audio system', 'mobile phone charger', 'broadband router', 'fan', 'radio', 'food processor', 'drill', 'computer monitor', 'USB hub'}


In [135]:
buildings = [1,2,5]
# len(ukdale.buildings[1].elec.meters)
metersPerBuilding = [53,19,5,6,25]
single_apps = [[],[],[]]

for i in range(3):
    for j in ukdale.buildings[i+1].elec.meters:
        if len(j.appliances) == 1:
            single_apps[i].append(j.appliances[0].type['type'])
print(set(single_apps[0])) # single apps aus Building 1
print(set(single_apps[1])) # single apps aus Building 2
print(set(single_apps[2])) # single apps aus Building 5

{'audio amplifier', 'clothes iron', 'light', 'coffee maker', 'oven', 'charger', 'boiler', 'fan', 'breadmaker', 'dish washer', 'baby monitor', 'computer monitor', 'hair straighteners', 'printer', 'audio system', 'active subwoofer', 'hair dryer', 'radio', 'fridge freezer', 'soldering iron', 'microwave', 'HTPC', 'desktop computer', 'tablet computer charger', 'mobile phone charger', 'vacuum cleaner', 'solar thermal pumping station', 'television'}
{'computer', 'fridge', 'cooker', 'toaster', 'laptop computer', 'active speaker', 'kettle', 'broadband router', 'modem', 'running machine', 'dish washer', 'external hard disk', 'games console', 'computer monitor', 'washing machine', 'microwave', 'rice cooker'}
{'electric space heater', 'laptop computer', 'projector', 'kettle'}


In [130]:
# common_single_apps = set(single_apps[0]).intersection(single_apps[1],single_apps[2]) # keine Übereinstimmungen
common_single_apps = set(single_apps[0]).intersection(single_apps[1]) # Übereinstimmungen von single apps aus Building 1 und 2
common_single_apps

{'computer monitor', 'dish washer', 'microwave'}

In [140]:
apps = common_single_apps.intersection(common_apps) # Übereinstimmungen von common und single apps aus Building 1 und 2 
apps

{'computer monitor'}

### Training Config

In [144]:
from nilmtk.api import API
from nilmtk.disaggregate import Mean
from nilmtk_contrib.disaggregate import DAE,Seq2Point, Seq2Seq, RNN

epochs = 30
batch_size = 32

config = {
    'power': {
        'mains': ['apparent','active'],
        'appliance': ['apparent','active']
    },
    'sample_rate': 10,

    'appliances': ['microwave', 'dish washer', 'washer dryer', 'coffee maker','computer monitor','laptop computer','audio system','light'],
    'methods': {
        'RNN':RNN({'n_epochs':epochs,'batch_size':batch_size}),
        'DAE':DAE({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Point':Seq2Point({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Seq':Seq2Seq({'n_epochs':epochs,'batch_size':batch_size}),
        'Mean': Mean({}),
    },
    'train': {
        'datasets': {
            'Dataport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-04',
                        'end_time': '2015-04-05'
                    },
                    2: {
                        'start_time': '2015-01-28',
                        'end_time': '2015-01-30'
                        },
                }
            }
        }
    },
    'test': {
        'datasets': {
            'Datport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-05',
                        'end_time': '2015-04-06'
                    },
                    2: {
                        'start_time': '2015-04-05',
                        'end_time': '2015-04-06'
                    },
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

Start Training

In [ ]:
api_res = API(config)